In [3]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPVisionModelWithProjection

model = CLIPVisionModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")
model.eval()
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [4]:
from transformers import AutoProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
model.eval()
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [84]:
from tqdm.auto import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import torch

# Load the colors
df_colors = pd.read_csv('HC_RGB.csv')

# Load the words
df_words = pd.read_excel("Words_Hues&Clues_Eng.xlsx", index_col=0)

# Create PIL images from RGB values
color_images = []
for index, row in df_colors.iterrows():
    rgb = (int(row['R']), int(row['G']), int(row['B']))
    img = Image.new('RGB', (256, 256), color=rgb)
    color_images.append(img)

In [86]:
results = defaultdict(list)
show_colors = True

# Iterate over all the words
it = 0
for word in tqdm(df_words.palabra.to_list()):
# for word in ["EGG", "LEMON"]:
    # Process the images and text
    inputs = processor(text=[word], images=color_images, return_tensors="pt", padding=True)

    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    text_embeds = outputs.text_embeds
    image_embeds = outputs.image_embeds

    # Normalize embeddings
    text_embeds /= text_embeds.norm(dim=-1, keepdim=True)
    image_embeds /= image_embeds.norm(dim=-1, keepdim=True)

    # Calculate cosine similarity
    cosine_sim = (text_embeds @ image_embeds.T).squeeze(0)

    # Get top 5 colors
    top5_indices = cosine_sim.argsort(descending=True)[:5]
    top5_distances = cosine_sim.sort(descending=True)[0][:5]

    # print(f"Top 5 colors for the word '{word}':")

    if show_colors: fig, axes = plt.subplots(1,5)
    for j, i in enumerate(top5_indices):
        color_info = df_colors.iloc[i.item()]
        color_info["distance"] = top5_distances[j].item()
        results[word].append(color_info.to_dict())
        # print(f"- Coords: ({color_info['coordenada_x']}, {color_info['coordenada_y']}), RGB: ({color_info['R']}, {color_info['G']}, {color_info['B']})")
        if show_colors:
            axes[j].imshow(color_images[i.item()])
            axes[j].axis("off")
            axes[j].set_title(f"({color_info['coordenada_x']}, {color_info['coordenada_y']})")
    if show_colors:
        plt.suptitle(word)
        plt.savefig(f"{word}.png", dpi=300)
        plt.close()
        # plt.show()
    it += 1
    # if it == 5: break

  0%|          | 0/100 [00:00<?, ?it/s]

/var/folders/fj/7ttqfxt94c5cqdh00xr61t880000gn/T/ipykernel_38010/992778214.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  color_info["distance"] = top5_distances[j].item()
/var/folders/fj/7ttqfxt94c5cqdh00xr61t880000gn/T/ipykernel_38010/992778214.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  color_info["distance"] = top5_distances[j].item()
/var/folders/fj/7ttqfxt94c5cqdh00xr61t880000gn/T/ipykernel_38010/992778214.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [87]:
flat_data = [
    {**record, 'word':word} 
    for word, records in results.items() 
    for record in records
]

# 2. Create the DataFrame
df = pd.DataFrame(flat_data)

# Optional: Reorder columns to put 'Animal' first
cols = ['word'] + [c for c in df.columns if c != 'word']
df = df[cols]

df.head()

,word,coordenada_x,coordenada_y,R,G,B,distance
0,SALMON,G,10,227,147,113,0.262502
1,SALMON,F,10,220,130,98,0.261921
2,SALMON,F,11,223,134,112,0.261548
3,SALMON,F,9,219,128,88,0.260856
4,SALMON,G,11,225,146,123,0.260832


In [88]:
df.to_csv("results_handc.csv", index=0)